In [166]:
import math
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

In [167]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

This model approximates the function f(x) = 1/1+e^-x

In [168]:
import struct

def int_to_64bit_list(number):
    binary_str = format(number, '064b')
    return [int(bit) for bit in binary_str]

def float_to_64bit_list(f):
    packed = struct.pack('!d', f)
    bits = ''.join(f'{byte:08b}' for byte in packed)
    return [int(bit) for bit in bits]

In [169]:
# test dataset
function = lambda x: math.sin(x)
ints = [x for x in range(1, 100_000)]

input_data = [int_to_64bit_list(x) for x in ints]

output_labels = [function(x) for x in ints]


In [170]:
t = torch.Tensor(input_data)
print(t[500])


t = torch.Tensor(output_labels)
print(t[500])

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 1., 1., 1., 1., 0., 1., 0., 1.])
tensor(-0.9965)


In [171]:
class Data(Dataset):
    def __init__(self, inputs, outputs):
        self.inputs = inputs
        self.labels = outputs

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return torch.tensor(self.inputs[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.float32)

In [172]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()  
        self.layer1 = nn.Linear(64, 64 * 5) # 64 bits in
        self.layer2 = nn.Linear(64 * 5, 64 * 10)  
        self.output = nn.Linear(64 * 10, 1) # 64 bits out

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = self.output(x)
        return x

In [181]:
training_dataset = Data(input_data, output_labels)
training_loader = DataLoader(training_dataset, batch_size=32, shuffle=True)

model = Model().to(device)

loss = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(50):
    for data in training_loader:
        inputs, labels = data
        labels = labels.view(-1, 1)
        optimizer.zero_grad()
        out = model(inputs.to(device))
        loss_size = loss(out, labels.to(device))
        loss_size.backward()
        optimizer.step()

    print(f"Loss: {loss_size}")

torch.save(model.state_dict(), "model")

Loss: 0.5203452706336975
Loss: 0.537867546081543
Loss: 0.5687859654426575
Loss: 0.5748502612113953
Loss: 0.477509081363678
Loss: 0.39628034830093384
Loss: 0.07464863359928131
Loss: 0.05374307185411453
Loss: 0.0495455376803875
Loss: 0.020435582846403122
Loss: 0.04044293239712715
Loss: 0.010281447321176529
Loss: 0.00950159877538681
Loss: 0.011789032258093357
Loss: 0.028798924759030342
Loss: 0.00361211271956563
Loss: 0.0035207923501729965
Loss: 0.007657609414309263
Loss: 0.004893044009804726
Loss: 0.006681011524051428
Loss: 0.010199647396802902
Loss: 0.001999662956222892
Loss: 0.010047278366982937
Loss: 0.016984721645712852
Loss: 0.006052306387573481
Loss: 0.004431175533682108
Loss: 0.0037989444099366665
Loss: 0.013352916575968266
Loss: 0.03390971198678017
Loss: 0.035503167659044266
Loss: 0.0015753427287563682
Loss: 0.001956128515303135
Loss: 0.026555636897683144
Loss: 0.0016077703330665827
Loss: 0.004002026282250881
Loss: 0.0019036629237234592
Loss: 0.002629634225741029
Loss: 0.003770967

In [184]:
with torch.no_grad():
  val = torch.tensor(input_data[400], dtype=torch.float32).to(device)
  answer = torch.tensor(output_labels[400], dtype=torch.float32).to(device)

  outputs = model(val)  # Get predictions from the model.
  print(outputs.data)
  print(answer)
# print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

tensor([-0.9194], device='mps:0')
tensor(-0.9018, device='mps:0')
